In [ ]:
!pip install simpledbf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import re, os
from pathlib import Path
from glob import glob
from simpledbf import Dbf5
import matplotlib.pyplot as plt
from statistics import mode

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import Ridge
from sklearn.metrics import log_loss

def Read():
  train = pd.read_csv(path_start/"input/df_train.csv")
  test = pd.read_csv(path_start/"input/test.csv")
  df_ConCat = pd.concat([train, test])
  df_ConCat = df_ConCat.drop(["Unnamed: 0", "地域", "前面道路：方位", "前面道路：種類",	"前面道路：幅員（ｍ）", 
                              "種類", "土地の形状", "間口", "延床面積（㎡）"], axis=1)
  return df_ConCat

path_start = Path("/content/drive/MyDrive/Nishika/Mansion_2023_spring")
path_import = path_start/"Inport_data/S12-21_GML"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_station = Dbf5(path_import/'utf8/S12-21_NumberOfPassengers.dbf', codec='utf8')
df = Read()
df_station = df_station.to_dataframe()
print(df_station.shape)
display(df_station.head())

(10484, 45)


,S12_001,S12_002,S12_003,S12_004,S12_005,S12_006,S12_007,S12_008,S12_009,S12_010,...,S12_036,S12_037,S12_038,S12_039,S12_040,S12_041,S12_042,S12_043,S12_044,S12_045
0,二月田,九州旅客鉄道,指宿枕崎線,11,2,1,3,NaN,0,1,...,NaN,658,1,1,NaN,690,1,1,NaN,318
1,古島,沖縄都市モノレール,沖縄都市モノレール線,23,5,1,1,NaN,3907,1,...,NaN,5086,1,1,NaN,5143,1,1,NaN,3404
2,お台場海浜公園,ゆりかもめ,東京臨海新交通臨海線,24,5,1,1,NaN,14612,1,...,NaN,17165,1,1,NaN,16899,1,1,NaN,7703
3,東京国際クルーズターミナル,ゆりかもめ,東京臨海新交通臨海線,24,5,1,1,NaN,3767,1,...,NaN,3551,1,1,NaN,3191,1,1,NaN,1383
4,テレコムセンター,ゆりかもめ,東京臨海新交通臨海線,24,5,1,1,NaN,12112,1,...,NaN,13475,1,1,NaN,12140,1,1,NaN,6536


S12_008 = 備考2011, 
S12_012 = 備考2012, 
S12_016 = 備考2013, 
S12_020 = 備考2014, 
S12_024 = 備考2015, 
S12_028 = 備考2016, 
S12_032 = 備考2017, 
S12_036 = 備考2018, 
S12_040 = 備考2019, 
S12_044 = 備考2020

In [ ]:
my_dict = {}
List_test = ["S12_{}".format(str(i).zfill(3)) for i in range(9,46) if i%4==1]
List_year = ["駅別乗降客数_{}".format(i+2011) for i in range(len(List_test))]

for i in range(len(List_test)):
    key = List_test[i]
    value = List_year[i]
    my_dict[key] = value
    
a = {"S12_001":"最寄駅：名称"}
a.update(my_dict)

In [ ]:
df_station = df_station.drop_duplicates(subset=['S12_001'])

df_a = df_station[["S12_001"] + List_test].rename(columns=a)
df_b = pd.merge(df, df_a, on="最寄駅：名称", how="left")
df_stage2 = df_b[["ID", "都道府県名"]+List_year]
display(df_stage2.shape)
df_stage2

(784713, 12)

,ID,都道府県名,駅別乗降客数_2011,駅別乗降客数_2012,駅別乗降客数_2013,駅別乗降客数_2014,駅別乗降客数_2015,駅別乗降客数_2016,駅別乗降客数_2017,駅別乗降客数_2018,駅別乗降客数_2019,駅別乗降客数_2020
0,8067540,茨城県,1661.0,1712.0,1670.0,1610.0,1514.0,1478.0,1488.0,1442.0,1458.0,1181.0
1,8027265,茨城県,11630.0,11698.0,12224.0,12878.0,12632.0,13314.0,13574.0,13946.0,13946.0,9914.0
2,8061526,茨城県,9800.0,10600.0,11900.0,12200.0,13000.0,13600.0,14100.0,14600.0,14600.0,9600.0
3,8086147,茨城県,31200.0,33300.0,34800.0,34300.0,35700.0,36900.0,35800.0,36400.0,36100.0,21000.0
4,8049498,茨城県,11361.0,12275.0,12889.0,13922.0,13973.0,15151.0,15630.0,14347.0,14638.0,10500.0
...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,沖縄県,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
784709,47006229,沖縄県,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
784710,47006331,沖縄県,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
784711,47006332,沖縄県,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 欠損値を埋める

直感だが、人口が一番多い東京都の影響で平均値が信用ならないと思うから、中央値を採用する

In [ ]:
df_stage3 = pd.DataFrame(df_stage2["ID"])
List_columns = list(df_stage2.columns)
List_columns.remove("ID")
List_columns.remove("都道府県名")

for aim in List_columns:
  df_stage3[aim] = df_stage2.groupby('都道府県名')[aim].transform(lambda x: x.fillna(x.median()))

df_stage3 = df_stage3.replace(0, 0.001)
df_stage3

,ID,駅別乗降客数_2011,駅別乗降客数_2012,駅別乗降客数_2013,駅別乗降客数_2014,駅別乗降客数_2015,駅別乗降客数_2016,駅別乗降客数_2017,駅別乗降客数_2018,駅別乗降客数_2019,駅別乗降客数_2020
0,8067540,1661.0,1712.0,1670.0,1610.0,1514.0,1478.0,1488.0,1442.0,1458.0,1181.0
1,8027265,11630.0,11698.0,12224.0,12878.0,12632.0,13314.0,13574.0,13946.0,13946.0,9914.0
2,8061526,9800.0,10600.0,11900.0,12200.0,13000.0,13600.0,14100.0,14600.0,14600.0,9600.0
3,8086147,31200.0,33300.0,34800.0,34300.0,35700.0,36900.0,35800.0,36400.0,36100.0,21000.0
4,8049498,11361.0,12275.0,12889.0,13922.0,13973.0,15151.0,15630.0,14347.0,14638.0,10500.0
...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,4193.0,4411.0,4656.0,4802.0,5008.0,5588.0,5961.0,6405.0,6394.0,2853.0
784709,47006229,4193.0,4411.0,4656.0,4802.0,5008.0,5588.0,5961.0,6405.0,6394.0,2853.0
784710,47006331,4193.0,4411.0,4656.0,4802.0,5008.0,5588.0,5961.0,6405.0,6394.0,2853.0
784711,47006332,4193.0,4411.0,4656.0,4802.0,5008.0,5588.0,5961.0,6405.0,6394.0,2853.0


## 2021年、2022年を推定する

2020～2022年はコロナによる影響がある

In [ ]:
for i in range(2020, 2022):
  tr_x = df_stage3.drop(["駅別乗降客数_{}".format(i)], axis=1)
  tr_y = df_stage3["駅別乗降客数_{}".format(i)]

  scaler = StandardScaler()
  tr_x = scaler.fit_transform(tr_x)

  model = Ridge()
  model.fit(tr_x, tr_y)

  df_stage3["駅別乗降客数_{}".format(i+1)] = model.predict(tr_x)

## 出力

In [ ]:
df_stage3.to_csv(path_start/"feature_value/df_駅別乗降客数.csv", index=False)
pd.read_csv(path_start/"feature_value/df_駅別乗降客数.csv")

,ID,駅別乗降客数_2011,駅別乗降客数_2012,駅別乗降客数_2013,駅別乗降客数_2014,駅別乗降客数_2015,駅別乗降客数_2016,駅別乗降客数_2017,駅別乗降客数_2018,駅別乗降客数_2019,駅別乗降客数_2020,駅別乗降客数_2021,駅別乗降客数_2022
0,8067540,1661.0,1712.0,1670.0,1610.0,1514.0,1478.0,1488.0,1442.0,1458.0,1181.0,2650.016238,2649.979853
1,8027265,11630.0,11698.0,12224.0,12878.0,12632.0,13314.0,13574.0,13946.0,13946.0,9914.0,11150.398986,11150.361234
2,8061526,9800.0,10600.0,11900.0,12200.0,13000.0,13600.0,14100.0,14600.0,14600.0,9600.0,11367.268249,11367.152223
3,8086147,31200.0,33300.0,34800.0,34300.0,35700.0,36900.0,35800.0,36400.0,36100.0,21000.0,26542.526677,26542.676872
4,8049498,11361.0,12275.0,12889.0,13922.0,13973.0,15151.0,15630.0,14347.0,14638.0,10500.0,11931.504146,11931.371705
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,4193.0,4411.0,4656.0,4802.0,5008.0,5588.0,5961.0,6405.0,6394.0,2853.0,5943.501831,5943.168520
784709,47006229,4193.0,4411.0,4656.0,4802.0,5008.0,5588.0,5961.0,6405.0,6394.0,2853.0,5943.502927,5943.169616
784710,47006331,4193.0,4411.0,4656.0,4802.0,5008.0,5588.0,5961.0,6405.0,6394.0,2853.0,5943.503462,5943.170151
784711,47006332,4193.0,4411.0,4656.0,4802.0,5008.0,5588.0,5961.0,6405.0,6394.0,2853.0,5943.503467,5943.170157


## おまけ

In [ ]:
"""
#欠損値を埋める
df_stage2 = df_stage2.fillna(-1)

#標準化
scaler = StandardScaler()
scaler.fit(df_stage2)

train_x = scaler.transform(df_stage2)

#主成分分析
pca = PCA(n_components=2)
pca.fit(train_x)

train_x = pca.transform(train_x)
plt.plot(train_x)

"""

'\n#欠損値を埋める\ndf_stage2 = df_stage2.fillna(-1)\n\n#標準化\nscaler = StandardScaler()\nscaler.fit(df_stage2)\n\ntrain_x = scaler.transform(df_stage2)\n\n#主成分分析\npca = PCA(n_components=2)\npca.fit(train_x)\n\ntrain_x = pca.transform(train_x)\nplt.plot(train_x)\n\n'